In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
plt.style.use('dark_background')
import time
from datetime import datetime
import multiprocessing as mp
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Getting the data and feature engineering

In [ ]:
#Creating the functiong that will get the data for us
'''def get_data_simple(league):
    
    files = [file for file in listdir(f'./../raw_data/{league}')]
    data = pd.DataFrame()


    for file in files:
        df = pd.read_csv(f'./../raw_data/{league}/'+file)
        data = pd.concat([data, df])
    return data'''

## Get_data5

In [ ]:
def get_data5(league1, league2=None, league3=None, league4=None, League5=None):
    
    files = [file for file in listdir(f'./../raw_data/{league1}')]
    data = pd.DataFrame()
    
    #For the case we have 2 leagues to concatenate:
    if league2:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        return data
    
    #For the case we have 3 leagues to concatenate:
    if league2 and league3:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        return data
    
    #For the case we have 4 leagues to concatenate:
    if league2 and league3 and league4:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])

        return data
    
    #For the case we have 5 leagues to concatenate:
    if league2 and league3 and league4 and league5:
        files1 = [file for file in listdir(f'./../raw_data/{league1}')]
        files2 = [file for file in listdir(f'./../raw_data/{league2}')]
        files3 = [file for file in listdir(f'./../raw_data/{league3}')]
        files4 = [file for file in listdir(f'./../raw_data/{league4}')]
        files5 = [file for file in listdir(f'./../raw_data/{league5}')]
        
        for file1 in files1:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file1)
            data = pd.concat([data, df])
        for file2 in files2:
            df = pd.read_csv(f'./../raw_data/{league2}/'+file2)
            data = pd.concat([data, df])
        for file3 in files3:
            df = pd.read_csv(f'./../raw_data/{league3}/'+file3)
            data = pd.concat([data, df])
        for file4 in files4:
            df = pd.read_csv(f'./../raw_data/{league4}/'+file4)
            data = pd.concat([data, df])
        for file5 in files5:
            df = pd.read_csv(f'./../raw_data/{league5}/'+file5)
            data = pd.concat([data, df])

        return data

In [ ]:
data = get_data(league1='turkey', league2='eredivisie', league3 = 'greece')

In [ ]:
data.shape

In [ ]:
def feature_engineering_old(data, b=20, binned=False):
    '''
    This function creates all the columns that will be needed to create the analysis 
    and return the dataframe with all this changes
    
    b is the number of bins that we want to work with. Our start value for b will be 20.
    '''
        
    # total number of goals = goals from the home team + goals from visiting team
    data['nb_goals']=data['FTHG']+data['FTAG']

    # boolean: true or false regarding whether they were more than 2.5 goals
    data['over_2.5_goals']=data['nb_goals']>2.5

    # boolean: true or false regarding whether they were less than 2.5 goals
    data['under_2.5_goals']=data['nb_goals']<2.5

    # payout of betting on over 2.5 goals: we get 0 if we lose the bet, we get the AvgC if we win the bet (AvgC = market average of the odds)
    data['payout_over_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

    # payout of betting on under 2.5 goals: we get 0 if we lose the bet, we get the AvgC if we win the bet (AvgC = market average of the odds)
    data['payout_under_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

    #payout UNDER 2.5 for PINACLE specifically
    data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['PC<2.5']

    #payout OVER 2.5 for PINACLE specifically
    data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['PC>2.5']

    #payout UNDER 2.5 for 365 specifically
    data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365C<2.5']

    #payout OVER 2.5 for 365 specifically
    data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365C>2.5']
    
    #Implied Probability OVER 2.5 goals for overall market (AvgC)
    data['Implied Probability >2.5']=1/data['AvgC>2.5']*100
    
    #Implied Probability UNDER 2.5 goals for overall market (AvgC)
    data['Implied Probability <2.5']=1/data['AvgC<2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE
    data['Implied Probability <2.5 pinacle']=1/data['PC<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE
    data['Implied Probability >2.5 pinacle']=1/data['PC>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365
    data['Implied Probability <2.5 365']=1/data['B365C<2.5']*100

    #Implied Probability OVER 2.5 goals for 365
    data['Implied Probability >2.5 365']=1/data['B365C>2.5']*100
    
    # Binning the implied probabilities with bins of 10
    if binned:
        b=b
        bins = np.arange(0, 101, int(100/b))
        bins = bins.tolist()

        data['binned >2.5'] = pd.cut(data['Implied Probability >2.5'], bins)
        data['binned <2.5'] = pd.cut(data['Implied Probability <2.5'], bins)
        data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)
        data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)
        data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)
        data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)
        
    #Cleaning the data
    #data = data.dropna(subset=['HomeTeam', 'AwayTeam'], how='any')
    data = data[~data['HomeTeam'].isna()]
    data = data[~data['AwayTeam'].isna()]
    data.drop(columns=['Referee', 'Unnamed: 105'], inplace=True)
    #data.dropna()
    
    return data

## Working with all the data

In [3]:
def get_data(league1, wall=False):
    
    if wall:
        data = pd.DataFrame()
        leagues = listdir(f'./../raw_data/')
        data = pd.DataFrame()
        for league in leagues:
            files = listdir(f'./../raw_data/{league}')
            for file in files:
                df = pd.read_csv((f'./../raw_data/{league}/'+file))
                data = pd.concat([data, df])

        return data
    
    else:
        files = [file for file in listdir(f'./../raw_data/{league1}')]
        data = pd.DataFrame()

        for file in files:
            df = pd.read_csv(f'./../raw_data/{league1}/'+file)
            data = pd.concat([data, df])

        return data

In [4]:
data = get_data('turkey', wall=True)

In [5]:
def feature_engineering(data, b=20, binned=False):
    '''
    This function creates all the columns that will be needed to create the analysis 
    and return the dataframe with all this changes
    
    b is the number of bins that we want to work with. Our start value for b will be 20.
        '''
    #------------------------Number of Goals, Over and Under -----------------------------------
    
    # total number of goals = goals from the home team + goals from visiting team
    data['nb_goals']=data['FTHG']+data['FTAG']

    # boolean: true or false regarding whether they were more than 2.5 goals
    data['over_2.5_goals']=data['nb_goals']>2.5

    # boolean: true or false regarding whether they were less than 2.5 goals
    data['under_2.5_goals']=data['nb_goals']<2.5
    
    #-----------------------------Payout Opening ----------------------------------------------
    
    # payout under 2.5 for Average OPENING odds
    data['payout_avg_under_2.5'] = data['under_2.5_goals']*data['Avg<2.5']

    # payout over 2.5 for Average OPENING odds
    data['payout_avg_over_2.5'] = data['over_2.5_goals']*data['Avg>2.5']

    #payout UNDER 2.5 for PINACLE specifically
    data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['P<2.5']

    #payout OVER 2.5 for PINACLE specifically
    data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['P>2.5']

    #payout UNDER 2.5 for 365 specifically
    data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365<2.5']

    #payout OVER 2.5 for 365 specifically
    data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365>2.5']
    
    #------------------------------Payout Closing --------------------------------------------
    
    # payout under 2.5 for Average CLOSING odds
    data['payout_avg_under_closing_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

    # payout over 2.5 for Average CLOSING odds
    data['payout_avg_over_closing_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

    #payout UNDER 2.5 for PINACLE closing ddds specifically
    data['payout_under_2.5_pinacle_closing'] = data['under_2.5_goals']*data['PC<2.5']

    #payout OVER 2.5 for PINACLE closing odds specifically
    data['payout_over_2.5_pinacle_closing'] = data['over_2.5_goals']*data['PC>2.5']

    #payout UNDER 2.5 for 365 closing odds specifically
    data['payout_under_2.5_365_closing'] = data['under_2.5_goals']*data['B365C<2.5']

    #payout OVER 2.5 for 365 closing odds specifically
    data['payout_over_2.5_365_closing'] = data['over_2.5_goals']*data['B365C>2.5']
    
    #-------------------------- Implied Probability Opening ----------------------------------------
    
    #Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
    data['Implied Probability <2.5 avg']=1/data['Avg<2.5']*100

    #Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
    data['Implied Probability >2.5 avg']=1/data['Avg>2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE
    data['Implied Probability <2.5 pinacle']=1/data['P<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE
    data['Implied Probability >2.5 pinacle']=1/data['P>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365
    data['Implied Probability <2.5 365']=1/data['B365<2.5']*100

    #Implied Probability OVER 2.5 goals for 365
    data['Implied Probability >2.5 365']=1/data['B365>2.5']*100
    
    #------------------------- Implied Probability Closing -----------------------------------
    
    #Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
    data['Implied Probability <2.5 avg closing']=1/data['AvgC<2.5']*100

    #Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
    data['Implied Probability >2.5 avg closing']=1/data['AvgC>2.5']*100

    #Implied Probability UNDER 2.5 goals for PINACLE closing odds
    data['Implied Probability <2.5 pinacle closing']=1/data['PC<2.5']*100

    #Implied Probability OVER 2.5 goals for PINACLE closing odds
    data['Implied Probability >2.5 pinacle closing']=1/data['PC>2.5']*100

    #Implied Probability UNDER 2.5 goals for 365 closing odds
    data['Implied Probability <2.5 365 closing']=1/data['B365C<2.5']*100

    #Implied Probability OVER 2.5 goals for 365 closing odds
    data['Implied Probability >2.5 365 closing']=1/data['B365C>2.5']*100
    
    #---------------------------- Binning IP Opening -------------------------------------

    b=b
    bins = np.arange(0, 101, int(100/b))
    bins = bins.tolist()

    #Binning UNDER 2.5 Average Market opening odds
    data['binned <2.5 avg'] = pd.cut(data['Implied Probability <2.5 avg'], bins)

    #Binning Over 2.5 Average Market opening odds
    data['binned >2.5 avg'] = pd.cut(data['Implied Probability >2.5 avg'], bins)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)

    #Binned OVER 2.5 Pinnacle
    data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)
    
    #----------------------------- Binning IP Closing ------------------------------------------------

    #Binning UNDER 2.5 Average Market closing odds
    data['binned <2.5 avg closing'] = pd.cut(data['Implied Probability <2.5 avg closing'], bins)

    #Binning OVER 2.5 Average Market closing odds
    data['binned >2.5 avg closing'] = pd.cut(data['Implied Probability >2.5 avg closing'], bins)

    #Binned UNDER 2.5 Pinnacle closing odds
    data['binned <2.5 pinacle closing'] = pd.cut(data['Implied Probability <2.5 pinacle closing'], bins)

    #Binned OVER 2.5 Pinnacle CLOSING odds
    data['binned >2.5 pinacle closing'] = pd.cut(data['Implied Probability >2.5 pinacle closing'], bins)

    #Binned UNDER 2.5 bet365 CLOSING odds
    data['binned <2.5 365 closing'] = pd.cut(data['Implied Probability <2.5 365 closing'], bins)

    #Binned OVER 2.5 bet365 CLOSING odds
    data['binned >2.5 365 closing'] = pd.cut(data['Implied Probability >2.5 365 closing'], bins)
    
    #---------------------------- Binning Odds Opening ----------------------------------------------------
    
    bins2 = [1, 1.5, 2, 3, 99999]

    #Binning UNDER 2.5 Average Market opening odds
    data['binned odds <2.5 avg'] = pd.cut(data['Avg<2.5'], bins2)

    #Binning Over 2.5 Average Market opening odds
    data['binned odds >2.5 avg'] = pd.cut(data['Avg>2.5'], bins2)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned odds <2.5 pinacle'] = pd.cut(data['P<2.5'], bins2)

    #Binned OVER 2.5 Pinnacle
    data['binned odds >2.5 pinacle'] = pd.cut(data['P>2.5'], bins2)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned odds <2.5 365'] = pd.cut(data['B365<2.5'], bins2)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned odds >2.5 365'] = pd.cut(data['B365>2.5'], bins2)
    
    #----------------------------- Binning Odds Closing ----------------------------------------------------------
    
    #Binning UNDER 2.5 Average Market opening odds
    data['binned odds <2.5 avg closing'] = pd.cut(data['AvgC<2.5'], bins2)

    #Binning Over 2.5 Average Market opening odds
    data['binned odds >2.5 avg closing'] = pd.cut(data['AvgC>2.5'], bins2)

    #Binned UNDER 2.5 Pinnacle opening odds
    data['binned odds <2.5 pinacle closing'] = pd.cut(data['PC<2.5'], bins2)

    #Binned OVER 2.5 Pinnacle
    data['binned odds >2.5 pinacle closing'] = pd.cut(data['PC>2.5'], bins2)

    #Binned UNDER 2.5 bet365 OPENING odds
    data['binned odds <2.5 365 closing'] = pd.cut(data['B365C<2.5'], bins2)

    #Binned OVER 2.5 bet365 OPENING odds
    data['binned odds >2.5 365 closing'] = pd.cut(data['B365C>2.5'], bins2)
    
    
    #----------------------------- Other Features from D3 ------------------------------------------------------
    
    data['Pin_pays_better_under_boolean'] = data['PC<2.5'] > data['AvgC<2.5']
    data['Pin_pays_better_under_difference'] = data['PC<2.5'] / data['AvgC<2.5']
    data['%vig_p'] = (1 - (1 / (1/data['PC>2.5'] + 1/data['PC<2.5'])))*100
    data['%vig_avg'] = (1 - (1 / (1/data['AvgC>2.5'] + 1/data['AvgC<2.5'])))*100
    data['PC<2.5_P_boolean'] = data['PC<2.5'] < data['P<2.5']
    data['PC<2.5_P_relative_diff'] = data['PC<2.5'] / data['P<2.5']
    
    #----------------------- Odds and probability of the home team scoring under 2.5 -------------------------------
    data['Date'] = pd.to_datetime(data['Date'])
    
    lst1 = []
    lst2 = []
    for i, team in enumerate(data['HomeTeam']):
        date = data['Date'].iloc[i]
        total = len(data[(data['HomeTeam'] == team) & (data['Date'] < date)])
        n_under_home = data[(data['HomeTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
        try:
            lst1.append(1/(n_under_home[1]/total))
            lst2.append(n_under_home[1]/total)
        except:
            lst1.append(np.nan)
            lst2.append(np.nan)

    data['odds_home_under'] = lst1
    data['prob_home_under'] = lst2
    
    
    
    #----------------------- Odds and probability of the away team scoring under 2.5 -------------------------------
    
    lst3 = []
    lst4  = []
    for i, team in enumerate(data['AwayTeam']):
        date = data['Date'].iloc[i]
        total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
        n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
        try:
            lst3.append(1/(n_under_away2[1] / total2))
            lst4.append(n_under_away2[1] / total2)
        except:
            lst3.append(np.nan)
            lst4.append(np.nan)

    data['odds_away_under'] = lst3
    data['prob_away_under'] = lst4
    
        #----------------------- Odds and probability of the home team scoring over 2.5 -------------------------------
    
    lst5 = []
    lst6 = []
    for i, team in enumerate(data['HomeTeam']):
        date = data['Date'].iloc[i]
        total = len(data[(data['HomeTeam'] == team) & (data['Date'] < date)])
        n_under_home = data[(data['HomeTeam'] == team) & (data['Date'] < date)]['over_2.5_goals'].value_counts()
        try:
            lst5.append(1/(n_under_home[1]/total))
            lst6.append(n_under_home[1]/total)
        except:
            lst5.append(np.nan)
            lst6.append(np.nan)

    data['odds_home_over'] = lst5
    data['prob_home_over'] = lst6
    
     #----------------------- Odds and probability of the away team scoring over 2.5 -------------------------------
    
    lst7 = []
    lst8  = []
    for i, team in enumerate(data['AwayTeam']):
        date = data['Date'].iloc[i]
        total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
        n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['over_2.5_goals'].value_counts()
        try:
            lst7.append(1/(n_under_away2[1] / total2))
            lst8.append(n_under_away2[1] / total2)
        except:
            lst7.append(np.nan)
            lst8.append(np.nan)

    data['odds_away_over'] = lst7
    data['prob_away_over'] = lst8
    
    # -------------------- binning the odds and probability of the home and away teams under 2.5 ----------------------
    if binned:

        #------- Probability -------

        #binning the probability of the home team to have a game of less than 2.5 score
        data['binned prob_home_under'] = pd.cut(data['prob_home_under']*100, bins)

        #binning the probability of the away team to have a game of less than 2.5 score
        data['binned prob_away_under'] = pd.cut(data['prob_away_under']*100, bins)

        #--------- Odds ------------
        binodds = [1, 1.25, 1.42, 1.5, 1.6, 1.8, 2, 2.2, 2.5, 2.8, 3.5, 4, 100]

        #binning the odds of the away team to have a game of less than 2.5 score
        data['binned odds_away_under'] = pd.cut(data['odds_away_under'], binodds)

        #binning the odds of the home team to have a game of less than 2.5 score
        data['binned odds_home_under'] = pd.cut(data['odds_away_under'], binodds)


        # -------------------- binning the odds and probability of the home and away teams over 2.5 ----------------------

        #------- Probability -------

        #binning the probability of the home team to have a game of less than 2.5 score
        data['binned prob_home_over'] = pd.cut(data['prob_home_over']*100, bins)

        #binning the probability of the away team to have a game of less than 2.5 score
        data['binned prob_away_over'] = pd.cut(data['prob_away_over']*100, bins)

        #--------- Odds ------------
        binodds = [1, 1.25, 1.42, 1.5, 1.6, 1.8, 2, 2.2, 2.5, 2.8, 3.5, 4, 100]

        #binning the odds of the away team to have a game of less than 2.5 score
        data['binned odds_away_over'] = pd.cut(data['odds_away_over'], binodds)

        #binning the odds of the home team to have a game of less than 2.5 score
        data['binned odds_home_over'] = pd.cut(data['odds_away_over'], binodds)


    #-------------------------- Creating the prob and odds of the game -----------------------------------------------
    
    #---------------- Under --------------
    '''the mean between the probability of the home team to have a score of under 2.5 and the probability 
    of the away team to do the same'''
    
    data[' '] = (data['odds_away_under'] +  data['odds_home_under']) / 2
    data['prob_game_under'] = (data['prob_away_under'] + data['prob_home_under']) / 2
    
    #---------------- Over -------------

    '''the mean between the probability of the home team to have a score of over 2.5 and the probability 
    of the away team to do the same'''
    
    data['odds_game_over'] = (data['odds_away_over'] +  data['odds_home_over']) / 2
    data['prob_game_over'] = (data['prob_away_over'] + data['prob_home_over']) / 2
    
    #-------------------------- OneHotEncoding the binned probabilities columns ------------------------------------------
    
    if binned:
        if b == 5:
            #-------------------- Under -----------------------
            data = data[~data['binned prob_home_under'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_under']])
            bins_encoded = ohe.transform(data[['binned prob_home_under']])
            data["0, 20"], data["20, 40"], data["40, 60"], data["60, 80"], data["80, 100"] = bins_encoded.T

            #-------------------- Over -----------------------
            data = data[~data['binned prob_home_over'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_over']])
            bins_encoded = ohe.transform(data[['binned prob_home_over']])
            data["0, 20"], data["20, 40"], data["40, 60"], data["60, 80"], data["80, 100"] = bins_encoded.T

        if b == 10:
            #-------------------- Under -----------------------
            data = data[~data['binned prob_home_under'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_under']])
            bins_encoded = ohe.transform(data[['binned prob_home_under']])
            data["0, 10"], data["10, 20"], data["20, 30"], data["30, 40"], data["40, 50"], data["50, 60"], \
            data["60, 70"], data["70, 80"], data["80, 90"], data["90, 100"] = bins_encoded.T

            #-------------------- Over -----------------------
            data = data[~data['binned prob_home_over'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_over']])
            bins_encoded = ohe.transform(data[['binned prob_home_over']])
            data["0, 10"], data["10, 20"], data["20, 30"], data["30, 40"], data["40, 50"], data["50, 60"], \
            data["60, 70"], data["70, 80"], data["80, 90"], data["90, 100"] = bins_encoded.T

        if b == 20:
            #-------------------- Under -----------------------
            data = data[~data['binned prob_home_under'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_under']])
            bins_encoded = ohe.transform(data[['binned prob_home_under']])
            data["0, 5"], data["5, 10"], data["10, 15"], data["15, 20"], data["20, 25"], data["25, 30"], \
            data["30, 35"], data["35, 40"], data["40, 45"], data["45, 50"], data["50, 55"], data["55, 60"], \
            data["60, 65"], data["65, 70"], data["70, 75"], data["75, 80"], data["80, 85"], data["85, 90"], \
            data["90, 95"], data["95, 100"]= bins_encoded.T

            #-------------------- Over -----------------------
            data = data[~data['binned prob_home_over'].isna()]
            ohe = OneHotEncoder(sparse=False)
            ohe.fit(data[['binned prob_home_over']])
            bins_encoded = ohe.transform(data[['binned prob_home_over']])
            data["0, 5"], data["5, 10"], data["10, 15"], data["15, 20"], data["20, 25"], data["25, 30"], \
            data["30, 35"], data["35, 40"], data["40, 45"], data["45, 50"], data["50, 55"], data["55, 60"], \
            data["60, 65"], data["65, 70"], data["70, 75"], data["75, 80"], data["80, 85"], data["85, 90"], \
            data["90, 95"], data["95, 100"]= bins_encoded.T

    #-------------------------- OneHotEncoding the binned odds columns ------------------------------------------
    
        #-------------------- Under -----------------------                                       
        data = data[~data['binned odds_away_under'].isna()]
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(data[['binned odds_away_under']])
        bins_encoded = ohe.transform(data[['binned odds_away_under']])
        data["1.0, 1.25"], data["1.25, 1.42"], data["1.42, 1.5"], data["1.5, 1.6"],\
        data["1.6, 1.8"], data["1.6, 1.8"], data["1.8, 2.0"], data["2.0, 2.2"], \
        data["2.2, 2.5"], data["2.5, 2.8"], data["2.8, 3.5"], data["3.5, 4.0"] = bins_encoded.T

        #-------------------- Over -----------------------
        data = data[~data['binned odds_away_over'].isna()]
        ohe = OneHotEncoder(sparse=False)
        ohe.fit(data[['binned odds_away_over']])
        bins_encoded = ohe.transform(data[['binned odds_away_over']])
        data["1.0, 1.25"], data["1.25, 1.42"], data["1.42, 1.5"], data["1.5, 1.6"],\
        data["1.6, 1.8"], data["1.6, 1.8"], data["1.8, 2.0"], data["2.0, 2.2"], \
        data["2.2, 2.5"], data["2.5, 2.8"], data["2.8, 3.5"], data["3.5, 4.0"] = bins_encoded.T
    
    #------------------------------------ Cleaning the data ---------------------------------------------------------
    
    #data = data.dropna(subset=['HomeTeam', 'AwayTeam'], how='any')
    data = data[~data['HomeTeam'].isna()]
    data = data[~data['AwayTeam'].isna()]
    data = data[~data['PC>2.5'].isna()]
    data.drop(columns=['Referee', 'Unnamed: 105'], inplace=True)
    #data.dropna()
    
    
    return data

In [6]:
data = feature_engineering(data, b=5, binned=True)

In [7]:
data.shape

(19080, 199)

In [10]:
data[['Date','HomeTeam','AwayTeam','odds_home_under','odds_away_under', 'odds_game_under', 'PC<2.5', 'nb_goals']]

KeyError: "['odds_game_under'] not in index"

In [ ]:
data[['Date','HomeTeam','AwayTeam','odds_home_over','odds_away_over','odds_home_under','odds_away_under', 'odds_game_over', 'odds_game_under', 'PC<2.5', 'AvgC<2.5','nb_goals']]

In [9]:
data['wp'] = (data['PC<2.5'] - data['odds_game_under'] >= 0) & (data['nb_goals'] < 3)

KeyError: 'odds_game_under'

In [ ]:
data['wp'].value_counts()

In [ ]:
data['pred_odds'] = (data['odds_game_over'] - data['odds_game_under'] >= 0) & (data['nb_goals'] < 3)

In [ ]:
data['pred_odds'].value_counts()

## CLT - Central Limit Theorem Analysis

In [ ]:
n = 100
N = 20000
means = [fdf_under_pinacle[fdf_under_pinacle['binned <2.5 pinacle'].astype(str)=='(45, 50]']['payout_under_2.5_pinacle'].sample(n, replace=True).mean() for i in range(N)]

sns.histplot(means, bins=20)

In [ ]:
mu = fdf_under_pinacle[fdf_under_pinacle['binned <2.5 pinacle'].astype(str)=='(45, 50]']['payout_under_2.5_pinacle'].mean()
sigma = fdf_under_pinacle[fdf_under_pinacle['binned <2.5 pinacle'].astype(str)=='(45, 50]']['payout_under_2.5_pinacle'].std()

In [ ]:
from scipy.stats import skew, kurtosis

# $CHALLENGIFY_BEGIN
index_df = ['mean', 'stdev', 'kurtosis', 'skewness']
theory = [mu, sigma/np.sqrt(n), 0, 0]
real_life = [np.mean(means), np.std(means), skew(means), kurtosis(means)]
comparison_df = pd.DataFrame(list(zip(theory,real_life)), 
                             columns = ["CLT Theory","Real Tips"],
                             index = index_df)
round(comparison_df,2)
# $CHALLENGIFY_END

## Analysing the numbers of a Buyuksehyr team

In [ ]:
#general numbers of the team

In [ ]:
buyu_df = data[(data['HomeTeam'] == 'Buyuksehyr') | (data['AwayTeam'] == 'Buyuksehyr')]
number_of_games = len(buyu_df)
total_goals = int(buyu_df[['nb_goals']].sum())
mean_goals_games = total_goals / number_of_games

In [ ]:
number_of_games, total_goals, mean_goals_games

In [ ]:
buyu_df['under_2.5_goals'].value_counts()[1]

In [ ]:
#Home numbers of the team

In [ ]:
buyu_home_df = buyu_df[buyu_df['HomeTeam'] == 'Buyuksehyr']
goals_as_home = int(buyu_home_df['FTHG'].sum())
total_goals_ishome = int(buyu_home_df[['nb_goals']].sum())
total_away_goals = int(buyu_home_df['FTAG'].sum())
total_home_victory = buyu_home_df['FTR'].value_counts()[0]
total_home_draws = buyu_home_df['FTR'].value_counts()[1]
total_home_losses = buyu_home_df['FTR'].value_counts()[2]
mean_home_goals_games = total_goals_ishome / 192

In [ ]:
buyu_home_df['under_2.5_goals'].value_counts()

In [ ]:
goals_as_home, total_away_goals, total_goals_ishome, mean_home_goals_games, total_home_victory, total_home_draws, total_home_losses

In [ ]:
buyu_home_df['FTHG'].value_counts()

In [ ]:
#Away numbers of the team

In [ ]:
buyu_away_df = buyu_df[buyu_df['AwayTeam'] == 'Buyuksehyr']

In [ ]:
goals_as_away = int(buyu_away_df['FTAG'].sum())
total_goals_isaway = int(buyu_away_df[['nb_goals']].sum())
total_thome_goals = int(buyu_away_df['FTHG'].sum())
total_away_victory = buyu_away_df['FTR'].value_counts()[0]
total_away_draws = buyu_away_df['FTR'].value_counts()[2]
total_away_losses = buyu_away_df['FTR'].value_counts()[1]
mean_away_goals_games = total_goals_isaway / 192

In [ ]:
buyu_away_df['under_2.5_goals'].value_counts()

In [ ]:
buyu_odd_away = 1/(109/192)    
buyu_odd_away

In [ ]:
goals_as_away, total_thome_goals, total_goals_isaway, mean_away_goals_games, total_away_victory, total_away_draws, total_away_losses

In [ ]:
buyu_away_df[['nb_goals']].value_counts()

In [ ]:
buyu_away_df['FTR'].value_counts()

## Creating the home and away under odds feature

In [ ]:
'''lst1 = []
lst2 = []
for i, team in enumerate(data['HomeTeam']):
    total = len(data[data['HomeTeam'] == team])
    n_under_home = data[data['HomeTeam'] == team]['under_2.5_goals'].value_counts()[1]
    lst1.append(1/(n_under_home/total))
    lst2.append(n_under_home/total)
data['odds_home_under'] = lst1
data['prob_home_under'] = lst2'''

In [ ]:
'''lst3 = []
lst4  = []
for i, team in enumerate(data['AwayTeam']):
    total2 = len(data[data['AwayTeam'] == team])
    n_under_away2 = data[data['AwayTeam'] == team]['under_2.5_goals'].value_counts()[1]
    lst3.append(1/(n_under_away2 / total2))
    lst4.append(n_under_away2 / total2)
data['odds_away_under'] = lst3
data['prob_away_under'] = lst4'''

In [ ]:
data.head(23)

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
date = data['Date'].iloc[21]
data[(data['HomeTeam'] == 'Goztep') & (data['Date'] < date)]
data[(data['HomeTeam'] == 'Goztep') & (data['Date'] < date)]['under_2.5_goals'].value_counts()

In [ ]:
lst1 = []
lst2 = []
for i, team in enumerate(data['HomeTeam']):
    date = data['Date'].iloc[i]
    total = len(data[(data['HomeTeam'] == team) & (data['Date'] < date)])
    n_under_home = data[(data['HomeTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
    try:
        lst1.append(1/(n_under_home[1]/total))
        lst2.append(n_under_home[1]/total)
    except:
        lst1.append(np.nan)
        lst2.append(np.nan)
        
data['odds_home_under'] = lst1
data['prob_home_under'] = lst2

In [ ]:
lst3 = []
lst4  = []
for i, team in enumerate(data['AwayTeam']):
    date = data['Date'].iloc[i]
    total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
    n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
    try:
        lst3.append(1/(n_under_away2[1] / total2))
        lst4.append(n_under_away2[1] / total2)
    except:
        lst3.append(np.nan)
        lst4.append(np.nan)
        
data['odds_away_under'] = lst3
data['prob_away_under'] = lst4

In [ ]:
# Does Pinnacle pays better than the market feature
data['PC<2.5_is_better_than_AvgC'] = data['PC<2.5']>data['AvgC<2.5']
data['PC>2.5_is_better_than_AvgC'] = data['PC>2.5']>data['AvgC>2.5']





16h06
# binned OVER 2.5 Pinacle
new_view = data.groupby('prob_away_under')['payout_under_2.5_pinacle_closing'].agg(['mean','count'])
new_view

In [ ]:
new_view = data.groupby('prob_away_under')['payout_under_2.5'].agg(['mean','count'])
new_view

In [ ]:
new_view = data.groupby('prob_home_under')['payout_under_2.5'].agg(['mean','count'])
new_view

In [ ]:
# I am binning the odds of Pinnacle with the bins_odds defined below for feature exploration
bins_odds = [1, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4,  2.5, 3, 999999]
bins_odds_2 = [1,2,999999]
binning = np.arange(0,101, int(100/20))
#Binned UNDER 2.5 Pinnacle opening odds
data['binned prob_home_under'] = pd.cut(data['prob_home_under']*100, binning)
#data['PC<2.5 low odds'] = data['PC<2.5']<2
#Binned OVER 2.5 Pinnacle opening odds
#data['binned PC>2.5'] = pd.cut(data['PC>2.5'], bins_odds)

In [ ]:
data['binned prob_home_under'] = pd.cut(data['prob_home_under']*100, binning)

data.groupby('binned prob_home_under')['payout_under_2.5'].agg(['mean','count'])

In [ ]:
data['binned prob_away_under'] = pd.cut(data['prob_away_under']*100, binning)

In [ ]:
data['binned prob_away_under'] = pd.cut(data['prob_away_under']*100, binning)
data.groupby('binned prob_away_under')['payout_under_2.5'].agg(['mean','count'])

In [ ]:
data.shape

## Creating the home and away over odds feature

In [ ]:
 #----------------------- Odds and probability of the away team scoring over 2.5 -------------------------------
    
lst7 = []
lst8  = []
for i, team in enumerate(data['AwayTeam']):
    date = data['Date'].iloc[i]
    total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
    n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['over_2.5_goals'].value_counts()
    try:
        lst7.append(1/(n_under_away2[1] / total2))
        lst8.append(n_under_away2[1] / total2)
    except:
        lst7.append(np.nan)
        lst8.append(np.nan)

data['odds_away_over'] = lst7
data['prob_away_over'] = lst8


In [ ]:
data

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])


In [ ]:
date = datetime(2022,8,2)

In [ ]:
data[data['Date'] < date]

In [ ]:
data[(data['HomeTeam'] == 'Bordeaux') & (data['AwayTeam'] == 'Nantes') & (data['Date'] < date)]['over_2.5_goals'].value_counts()[0]

In [ ]:
data[(data['HomeTeam'] == 'Bordeaux') & (data['AwayTeam'] == 'Nantes') & (data['Date'] < date)]

In [ ]:
data[(data['HomeTeam'] == hometeam) & (data['AwayTeam'] == awayteam) & (data['Date'] < date)]['over_2.5_goals'].value_counts()[0]

In [ ]:
lst9 = []
lst10 = []
for i in range(len(data)):
    date = data['Date'].iloc[i]
    for hometeam in data['HomeTeam'].unique():
        for awayteam in data['AwayTeam'].unique():
            total2 = len(data[(data['HomeTeam'] == hometeam) & (data['AwayTeam'] == awayteam) & (data['Date'] < date)])
            n_under_away2 = data[(data['HomeTeam'] == hometeam) & (data['AwayTeam'] == awayteam) & (data['Date'] < date)]['over_2.5_goals'].value_counts()[0]
            try:
                lst9.append(1/(n_under_away2[1] / total2))
                lst10.append(n_under_away2[1] / total2)
            except:
                lst9.append(np.nan)
                lst10.append(np.nan)

data['odds_game_home_under'] = lst9
data['prob_game_home_under'] = lst10
        
    

In [ ]:
for i in range(len(data)):
    date = data['Date'].iloc[i]
    print(date)

In [ ]:
for awayteam in data['AwayTeam'].unique():
    for hometeam in data['HomeTeam'].unique():
        for j in enumerate(tqdm(data[data['AwayTeam'] == awayteam])):
            for i in enumerate(tqdm(data[data['HomeTeam'] == hometeam])):
                date = data['Date'].iloc[i]
                total2 = len(data[(data['HomeTeam'] == hometeam) & (data['AwayTeam'] == awayteam) & (data['Date'] < date)])
                n_under_away2 = data[(data['HomeTeam'] == hometeam) & (data['AwayTeam'] == awayteam) & (data['Date'] < date)]
                try:
                    lst9.append(1/(n_under_away2[1] / total2))
                    lst10.append(n_under_away2[1] / total2)
                except:
                    lst9.append(np.nan)
                    lst10.append(np.nan)

data['odds_away_over'] = lst9
data['prob_away_over'] = lst10

In [ ]:
lst9 = []
lst10 = []


for i, team1 in enumerate(tqdm(data['AwayTeam'])):
    for i, team2 in enumerate(tqdm(data['HomeTeam'])):
        date = data['Date'].iloc[i]
        total2 = len(data[(data['HomeTeam'] == team2) & (data['AwayTeam'] == team1) & (data['Date'] < date)])
        n_under_away2 = data[(data['HomeTeam'] == team2) & (data['AwayTeam'] == team1) & (data['Date'] < date)]
        try:
            lst9.append(1/(n_under_away2[1] / total2))
            lst10.append(n_under_away2[1] / total2)
        except:
            lst9.append(np.nan)
            lst10.append(np.nan)

data['odds_away_over'] = lst9
data['prob_away_over'] = lst10

In [ ]:
total2 = len(data[(data['HomeTeam'] == 'Bordeaux') & (data['AwayTeam'] == 'Nantes') & (data['Date'] < date)])
total2

In [ ]:
with mp.Pool(mp.cpu_count()) as pool:
    df['newcol'] = pool.map(f, df['col'])